In [1]:
!pip config set global.index-url https://mirrors.aliyun.com/pypi/simple/
!python -m pip install -U pip
!pip -V
!pip install sklearn pandas

Writing to /root/.config/pip/pip.conf
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
pip 20.2.4 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

In [3]:
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.3.1
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.2
numpy 1.18.5
pandas 1.1.4
sklearn 0.23.2
tensorflow 2.3.1
tensorflow.keras 2.4.0


In [4]:
# tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus))

logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(logical_gpus))

1
1


In [5]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_train_all = x_train_all / 255.0
x_test = x_test / 255.0
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28,1 )
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

In [7]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 256
train_dataset = make_dataset(x_train_scaled, y_train, 1, batch_size)
valid_dataset = make_dataset(x_valid_scaled, y_valid, 1, batch_size)

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 512)         1

In [12]:
# customized training loop.
# 1. define losses functions
# 2. define function train_step
# 3. define function test_step
# 4. for-loop trianing loop

loss_func = keras.losses.SparseCategoricalCrossentropy(
    reduction=keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
test_loss = keras.metrics.Mean(name='test_loss')
train_accuray = keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')
test_accuray = keras.metrics.SparseCategoricalAccuracy(
    name='test_accuracy')

optimizer = keras.optimizers.SGD(lr=0.01)

@tf.function
def train_step(inputs):
    images, labels = inputs
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuray.update_state(labels, predictions)
    return loss

@tf.function
def test_step(inputs):
    images, labels = inputs
    predictions = model(images)
    t_loss = loss_func(labels, predictions)
    test_loss.update_state(t_loss)
    test_accuray.update_state(labels, predictions)

epochs = 10
for epoch in range(epochs):
    total_loss = 0.
    num_batches = 0
    for x in train_dataset:
        start_time = time.time()
        total_loss += train_step(x)
        run_time = time.time() - start_time
        num_batches += 1
        print('\ntotal: %3.3f, num_batches: %d, '
              'average: %3.3f, time: %3.3f' 
              % (total_loss, num_batches, 
                 total_loss / num_batches, run_time), 
              end='')
    train_loss = total_loss / num_batches
    for x in valid_dataset:
        test_step(x)
    
    print('\rEpoch: %d, Loss: %3.3f, Acc: %3.3f,'
          'Val_Loss: %3.3f, Val_Acc: %3.3f' 
          % (epoch + 1, train_loss, train_accuray.result(),
            test_loss.result(), test_accuray.result()))
    test_loss.reset_states()
    train_accuray.reset_states()
    test_accuray.reset_states()


total: 0.256, num_batches: 1, average: 0.256, time: 0.301
total: 0.516, num_batches: 2, average: 0.258, time: 0.021
total: 0.815, num_batches: 3, average: 0.272, time: 0.020
total: 1.108, num_batches: 4, average: 0.277, time: 0.020
total: 1.362, num_batches: 5, average: 0.272, time: 0.021
total: 1.735, num_batches: 6, average: 0.289, time: 0.020
total: 2.110, num_batches: 7, average: 0.301, time: 0.022
total: 2.500, num_batches: 8, average: 0.313, time: 0.022
total: 2.792, num_batches: 9, average: 0.310, time: 0.020
total: 3.087, num_batches: 10, average: 0.309, time: 0.022
total: 3.361, num_batches: 11, average: 0.306, time: 0.020
total: 3.616, num_batches: 12, average: 0.301, time: 0.020
total: 3.850, num_batches: 13, average: 0.296, time: 0.021
total: 4.130, num_batches: 14, average: 0.295, time: 0.022
total: 4.461, num_batches: 15, average: 0.297, time: 0.018
total: 4.761, num_batches: 16, average: 0.298, time: 0.016
total: 5.093, num_batches: 17, average: 0.300, time: 0.016
total

total: 41.410, num_batches: 138, average: 0.300, time: 0.022
total: 41.724, num_batches: 139, average: 0.300, time: 0.025
total: 42.054, num_batches: 140, average: 0.300, time: 0.022
total: 42.356, num_batches: 141, average: 0.300, time: 0.020
total: 42.624, num_batches: 142, average: 0.300, time: 0.022
total: 42.886, num_batches: 143, average: 0.300, time: 0.021
total: 43.165, num_batches: 144, average: 0.300, time: 0.023
total: 43.434, num_batches: 145, average: 0.300, time: 0.024
total: 43.737, num_batches: 146, average: 0.300, time: 0.021
total: 44.044, num_batches: 147, average: 0.300, time: 0.020
total: 44.330, num_batches: 148, average: 0.300, time: 0.022
total: 44.572, num_batches: 149, average: 0.299, time: 0.021
total: 44.823, num_batches: 150, average: 0.299, time: 0.026
total: 45.146, num_batches: 151, average: 0.299, time: 0.021
total: 45.501, num_batches: 152, average: 0.299, time: 0.022
total: 45.721, num_batches: 153, average: 0.299, time: 0.022
total: 46.044, num_batch

total: 16.970, num_batches: 58, average: 0.293, time: 0.021
total: 17.276, num_batches: 59, average: 0.293, time: 0.020
total: 17.621, num_batches: 60, average: 0.294, time: 0.020
total: 17.888, num_batches: 61, average: 0.293, time: 0.020
total: 18.251, num_batches: 62, average: 0.294, time: 0.021
total: 18.602, num_batches: 63, average: 0.295, time: 0.022
total: 18.906, num_batches: 64, average: 0.295, time: 0.024
total: 19.189, num_batches: 65, average: 0.295, time: 0.021
total: 19.527, num_batches: 66, average: 0.296, time: 0.019
total: 19.773, num_batches: 67, average: 0.295, time: 0.023
total: 20.078, num_batches: 68, average: 0.295, time: 0.022
total: 20.342, num_batches: 69, average: 0.295, time: 0.019
total: 20.600, num_batches: 70, average: 0.294, time: 0.022
total: 20.907, num_batches: 71, average: 0.294, time: 0.023
total: 21.195, num_batches: 72, average: 0.294, time: 0.022
total: 21.500, num_batches: 73, average: 0.295, time: 0.024
total: 21.839, num_batches: 74, average:

total: 56.215, num_batches: 194, average: 0.290, time: 0.020
total: 56.509, num_batches: 195, average: 0.290, time: 0.020
total: 56.778, num_batches: 196, average: 0.290, time: 0.023
total: 57.100, num_batches: 197, average: 0.290, time: 0.023
total: 57.456, num_batches: 198, average: 0.290, time: 0.023
total: 57.756, num_batches: 199, average: 0.290, time: 0.021
total: 58.049, num_batches: 200, average: 0.290, time: 0.026
total: 58.310, num_batches: 201, average: 0.290, time: 0.023
total: 58.591, num_batches: 202, average: 0.290, time: 0.022
total: 58.877, num_batches: 203, average: 0.290, time: 0.023
total: 59.193, num_batches: 204, average: 0.290, time: 0.019
total: 59.520, num_batches: 205, average: 0.290, time: 0.020
total: 59.844, num_batches: 206, average: 0.291, time: 0.020
total: 60.047, num_batches: 207, average: 0.290, time: 0.022
total: 60.309, num_batches: 208, average: 0.290, time: 0.020
total: 60.550, num_batches: 209, average: 0.290, time: 0.021
total: 60.792, num_batch

total: 32.598, num_batches: 115, average: 0.283, time: 0.020
total: 32.843, num_batches: 116, average: 0.283, time: 0.020
total: 33.144, num_batches: 117, average: 0.283, time: 0.021
total: 33.410, num_batches: 118, average: 0.283, time: 0.021
total: 33.638, num_batches: 119, average: 0.283, time: 0.015
total: 33.905, num_batches: 120, average: 0.283, time: 0.015
total: 34.163, num_batches: 121, average: 0.282, time: 0.019
total: 34.431, num_batches: 122, average: 0.282, time: 0.017
total: 34.638, num_batches: 123, average: 0.282, time: 0.021
total: 34.873, num_batches: 124, average: 0.281, time: 0.021
total: 35.186, num_batches: 125, average: 0.281, time: 0.023
total: 35.486, num_batches: 126, average: 0.282, time: 0.022
total: 35.817, num_batches: 127, average: 0.282, time: 0.016
total: 36.066, num_batches: 128, average: 0.282, time: 0.021
total: 36.364, num_batches: 129, average: 0.282, time: 0.018
total: 36.589, num_batches: 130, average: 0.281, time: 0.024
total: 36.880, num_batch

...

total: 54.843, num_batches: 209, average: 0.262, time: 0.021
total: 55.089, num_batches: 210, average: 0.262, time: 0.016
total: 55.386, num_batches: 211, average: 0.262, time: 0.014
total: 55.639, num_batches: 212, average: 0.262, time: 0.022
total: 55.904, num_batches: 213, average: 0.262, time: 0.021
total: 56.154, num_batches: 214, average: 0.262, time: 0.022
Epoch: 7, Loss: 0.262, Acc: 0.903,Val_Loss: 0.291, Val_Acc: 0.893

total: 0.273, num_batches: 1, average: 0.273, time: 0.026
total: 0.566, num_batches: 2, average: 0.283, time: 0.024
total: 0.811, num_batches: 3, average: 0.270, time: 0.022
total: 1.151, num_batches: 4, average: 0.288, time: 0.021
total: 1.359, num_batches: 5, average: 0.272, time: 0.023
total: 1.610, num_batches: 6, average: 0.268, time: 0.019
total: 1.815, num_batches: 7, average: 0.259, time: 0.022
total: 2.028, num_batches: 8, average: 0.253, time: 0.023
total: 2.218, num_batches: 9, average: 0.246, time: 0.016
total: 2.459, num_batches: 10, average: 0.246

total: 33.705, num_batches: 131, average: 0.257, time: 0.022
total: 33.973, num_batches: 132, average: 0.257, time: 0.015
total: 34.159, num_batches: 133, average: 0.257, time: 0.023
total: 34.412, num_batches: 134, average: 0.257, time: 0.020
total: 34.663, num_batches: 135, average: 0.257, time: 0.022
total: 34.862, num_batches: 136, average: 0.256, time: 0.019
total: 35.181, num_batches: 137, average: 0.257, time: 0.021
total: 35.514, num_batches: 138, average: 0.257, time: 0.021
total: 35.802, num_batches: 139, average: 0.258, time: 0.021
total: 36.095, num_batches: 140, average: 0.258, time: 0.023
total: 36.341, num_batches: 141, average: 0.258, time: 0.019
total: 36.595, num_batches: 142, average: 0.258, time: 0.025
total: 36.837, num_batches: 143, average: 0.258, time: 0.019
total: 37.104, num_batches: 144, average: 0.258, time: 0.025
total: 37.350, num_batches: 145, average: 0.258, time: 0.024
total: 37.574, num_batches: 146, average: 0.257, time: 0.021
total: 37.792, num_batch

total: 13.186, num_batches: 52, average: 0.254, time: 0.023
total: 13.391, num_batches: 53, average: 0.253, time: 0.022
total: 13.601, num_batches: 54, average: 0.252, time: 0.020
total: 13.843, num_batches: 55, average: 0.252, time: 0.020
total: 14.158, num_batches: 56, average: 0.253, time: 0.021
total: 14.386, num_batches: 57, average: 0.252, time: 0.022
total: 14.574, num_batches: 58, average: 0.251, time: 0.020
total: 14.874, num_batches: 59, average: 0.252, time: 0.023
total: 15.153, num_batches: 60, average: 0.253, time: 0.023
total: 15.392, num_batches: 61, average: 0.252, time: 0.020
total: 15.620, num_batches: 62, average: 0.252, time: 0.020
total: 15.864, num_batches: 63, average: 0.252, time: 0.023
total: 16.085, num_batches: 64, average: 0.251, time: 0.022
total: 16.344, num_batches: 65, average: 0.251, time: 0.023
total: 16.671, num_batches: 66, average: 0.253, time: 0.023
total: 17.015, num_batches: 67, average: 0.254, time: 0.020
total: 17.349, num_batches: 68, average:

total: 47.451, num_batches: 188, average: 0.252, time: 0.023
total: 47.743, num_batches: 189, average: 0.253, time: 0.024
total: 48.070, num_batches: 190, average: 0.253, time: 0.026
total: 48.311, num_batches: 191, average: 0.253, time: 0.023
total: 48.589, num_batches: 192, average: 0.253, time: 0.022
total: 48.790, num_batches: 193, average: 0.253, time: 0.022
total: 48.999, num_batches: 194, average: 0.253, time: 0.020
total: 49.261, num_batches: 195, average: 0.253, time: 0.023
total: 49.588, num_batches: 196, average: 0.253, time: 0.017
total: 49.797, num_batches: 197, average: 0.253, time: 0.023
total: 50.023, num_batches: 198, average: 0.253, time: 0.021
total: 50.232, num_batches: 199, average: 0.252, time: 0.018
total: 50.502, num_batches: 200, average: 0.253, time: 0.027
total: 50.740, num_batches: 201, average: 0.252, time: 0.023
total: 51.054, num_batches: 202, average: 0.253, time: 0.020
total: 51.310, num_batches: 203, average: 0.253, time: 0.021
total: 51.574, num_batch

total: 27.025, num_batches: 111, average: 0.243, time: 0.022
total: 27.299, num_batches: 112, average: 0.244, time: 0.021
total: 27.455, num_batches: 113, average: 0.243, time: 0.020
total: 27.631, num_batches: 114, average: 0.242, time: 0.017
total: 27.868, num_batches: 115, average: 0.242, time: 0.021
total: 28.079, num_batches: 116, average: 0.242, time: 0.021
total: 28.355, num_batches: 117, average: 0.242, time: 0.021
total: 28.668, num_batches: 118, average: 0.243, time: 0.016
total: 29.022, num_batches: 119, average: 0.244, time: 0.020
total: 29.214, num_batches: 120, average: 0.243, time: 0.024
total: 29.493, num_batches: 121, average: 0.244, time: 0.021
total: 29.832, num_batches: 122, average: 0.245, time: 0.021
total: 30.048, num_batches: 123, average: 0.244, time: 0.021
total: 30.272, num_batches: 124, average: 0.244, time: 0.021
total: 30.508, num_batches: 125, average: 0.244, time: 0.021
total: 30.792, num_batches: 126, average: 0.244, time: 0.023
total: 31.017, num_batch